# Treasury Yield Tracker

## Introduction

The project automates the process of downloading Treasury rate data and displaying the past interest rate since year-end 2022.  
It consists of sections for installing necessary libraries to download the Treasury rate data from FRED, preprocessing the data for visualization, and creating a line plot to visualize the data.

## Requirements and Import

In [89]:
from fredapi import Fred
import pandas as pd
import altair as alt
import numpy as np

In [90]:
print("pandas:", pd.__version__)
print("altair:", alt.__version__)
print("numpy:", np.__version__)

pandas: 2.2.3
altair: 5.5.0
numpy: 1.26.4


In [91]:
import os
# Load from secrets
fred_api_key = os.getenv("FRED_API_KEY")
fred = Fred(api_key=fred_api_key)

## Download Treasury rate data

In [92]:
# Define series to download
series_ids = {
    "3-Month": "DGS3MO",
    "1-Year": "DGS1",
    "5-Year": "DGS5",
    "10-Year": "DGS10",
    "30-Year": "DGS30"
}

# Download each series and merge into one DataFrame
df_list = []
for label, code in series_ids.items():
    series = fred.get_series(code)
    df = series.rename(label).to_frame()
    df_list.append(df)

# Merge on index (i.e., Date)
combined_df = pd.concat(df_list, axis=1)
combined_df.index.name = "Date"
combined_df = combined_df.dropna(how="all")  # Drop rows where all yields are NaN


In [93]:
# Reset index to make 'Date' a column
combined_df = combined_df.reset_index()

In [94]:
print(combined_df.tail())  # For Jupyter or console
print(combined_df.head())  # For Jupyter or console

#st.dataframe(combined_df.tail())  # For Streamlit display

            Date  3-Month  1-Year  5-Year  10-Year  30-Year
15818 2025-05-02     4.33    4.00    3.92     4.33     4.79
15819 2025-05-05     4.33    4.02    3.95     4.36     4.83
15820 2025-05-06     4.33    3.98    3.90     4.30     4.81
15821 2025-05-07     4.34    4.00    3.87     4.26     4.77
15822 2025-05-08     4.34    4.05    4.00     4.37     4.83
        Date  3-Month  1-Year  5-Year  10-Year  30-Year
0 1962-01-02      NaN    3.22    3.88     4.06      NaN
1 1962-01-03      NaN    3.24    3.87     4.03      NaN
2 1962-01-04      NaN    3.24    3.86     3.99      NaN
3 1962-01-05      NaN    3.26    3.89     4.02      NaN
4 1962-01-08      NaN    3.31    3.91     4.03      NaN


## Visualize the data

In [95]:
# Melt for long format
melted_df = combined_df.melt(id_vars='Date', var_name='Ticker', value_name='Yield')

# Filter data to only keep records from September 2022 onwards
melted_df = melted_df[melted_df['Date'] >= '2022-09-01']

# Remove NaN yields (some maturities are missing on certain dates)
melted_df.dropna(subset=["Yield"], inplace=True)

# Calculate y-axis bounds
min_y_value = melted_df['Yield'].min()
max_y_value = melted_df['Yield'].max()
y_start = np.floor(min_y_value * 2) / 2
y_end = np.ceil(max_y_value * 2) / 2

# Sorting list for legend and line color order
output_list = list(series_ids.keys())

# Create Altair line chart
chart = alt.Chart(melted_df).mark_line().encode(
    x=alt.X('Date:T', title="Date"),
    y=alt.Y('Yield:Q', scale=alt.Scale(domain=[y_start, y_end]), title="Yield (%)"),
    color=alt.Color('Ticker:N', sort=output_list),
    tooltip=['Date:T', 'Ticker:N', 'Yield:Q']
).properties(
    width=700,
    height=400,
    title='Daily Treasury Yields Over Time'
)

# Streamlit output
#st.altair_chart(chart, use_container_width=True)
# Draw the line
chart

alt.Chart(...)